## Purpose

This file is designed to construct our analysis sample. This notebook extracts CVD outcome data from the ICD records,
then the first 10 years of exposure data and constructs rates per 100 for each individual per year and averages by the
age of 1, 5 and 10. It also constructs fixed effects based on September-August year to control for seasonality of the 
disease better. Finally, the notebook mergers in variables needed from the UK Biobank itself, such as gender,
fluid intelligence and education years.

In [18]:
from miscSupports import load_json, terminal_time, load_yaml, validate_path, sep_num, rebase_year
from weightGIS.weighting.Calculate import assigned_exposure
from ICDBioAssign.core import ICDBioAssign
from csvObject import CsvObject, write_csv
from IPython.display import clear_output

from pyBlendFigures.Constructors import Prisma
from pyBlendFigures import BlendFigure

from datetime import datetime
from pathlib import Path
import os

# This is the relative path to the env.yaml which contains the following paths
env = load_yaml(Path(Path(os.path.abspath('')).parent, "env.yaml"))

# Directory for common extracted files from the UK Biobank which may have use for multiple papers.
# For this paper it contains
#   ICD 9 and 10 extractions
#   ID's of UK Biobank participants who have withdrawn from the study to be removed
#   Gender of participants
CONSTRUCTION_DIRECTORY = validate_path(env["CONSTRUCT_DIR"])

# Directory for this papers sensitive data which contains UK Biobank information
PROJECT_DIRECTORY = validate_path(env["PROJECT_DIR"])

# Root Directory for the BIO-HGIS databases
DATABASE_DIRECTORY = validate_path(env["BIO_HGIS_DIR"])

# External directory that contains ICD defintions
EXTERNAL_DIRECTORY = validate_path(env["EXTERNAL_DIR"])

# Blend Figure directory
BLEND_FIG_DIR = validate_path(env["BLEND_FIGURE_DIR"])

# BIO-HGIS extractions
# These represent the column indexes for the linked BIO-HGIS UK Biobank locations of birth. It also contains the year
# and month of birth which we use to extract from
year_i = 1
month_i = 2
district_id = 8
district_i = 9
district_type_i = 10
nation_i = 31

# Initialise the PRISMA plot object of withdrawals and the blend figure constructor.
# BLEND_PATH is the root to the blender.exe
# BLEND_FIGURE_DIR is the root of the folder that contains the figures made by blender
figure = BlendFigure(validate_path(env["BLEND_PATH"]), BLEND_FIG_DIR)
prisma_obj = Prisma(30)

print(f"Setup complete: All paths valid. {terminal_time()}")

Setup complete: All paths valid. 15:13


### Isolating Cardiovascular disease

A CVD definition has already been constructed and well-defined by working alongside clinicians by
[Carter et al (2019)][carter] which we extend to also include rheumatism as our control. Here we use the definitions 
within the *External Files* directory to construct phenotypic values via ICDBioAssign.

In [ ]:
icd = ICDBioAssign(Path(EXTERNAL_DIRECTORY, "Definitions.csv"), PROJECT_DIRECTORY, "CVD_Outcomes")

icd.set_definitions(Path(CONSTRUCTION_DIRECTORY, "ICD10.csv"))
icd.set_definitions(Path(CONSTRUCTION_DIRECTORY, "ICD9.csv"), icd_10=False)
icd.set_definitions(Path(CONSTRUCTION_DIRECTORY, "CoD.csv"))
icd.compile_and_write()
clear_output(True)
print(f"Constructed CVD Phenotypic values {terminal_time()}")

## Load ID data
We load the regionally assigned individuals based on the 1951 census locations from vision of britain

In [19]:
id_data = CsvObject(Path(DATABASE_DIRECTORY, "1951_weighted_birth_locations.csv"))
print(f"Loaded weighted data into memory {terminal_time()}")

Loaded weighted data into memory 15:13


### Remove withdrawn Id's

The Uk Biobank frequently removes individuals whom have withdrawn consent, here we remove individuals whom are not
to be included before starting formally one the analysis and prisma plot. 

In [20]:
removal_ids = CsvObject(Path(CONSTRUCTION_DIRECTORY, "UKB_Withdrawal_IDs.csv"), set_columns=True)
app_data = [row for row in id_data.row_data if row[0] not in removal_ids[0]]

print(f"Original {id_data.column_length} has been successfully reduced to {len(app_data)}\n"
      f"Removed {id_data.column_length - len(app_data)}")

Original 502507 has been successfully reduced to 502488
Removed 19


### Clean our sample

There are lots of individuals whom are within the sample that we cannot use for our analysis. We need to be clear in the
reasoning of the removal of these individuals from the sample, to make it clear that we are not purposely biasing our 
results through such exclusions.

In [21]:
# Set base number of observations
prisma_obj.add_element(f"UK Biobank Population\n\nN = {sep_num(len(app_data))}", 0, 0)

# We have no data for scotland, so remove them
ids_england_wales = [row for row in app_data if row[nation_i] != "SCOTLAND"]
prisma_obj.add_element(f"Born in Scotland\n\nN = {sep_num(len(app_data) - len(ids_england_wales))}", 1, 1)
prisma_obj.add_element(f"UK Biobank Population not in Scotland\n\nN = {sep_num(len(ids_england_wales))}", 0, 2)

# Individuals with no birth location cannot be used
ids_birth_location = [row for row in ids_england_wales if row[district_i] != "No or invalid coordinates"]
prisma_obj.add_element(f"No or invalid birth coordinates\n\nN = {sep_num(len(ids_england_wales) - len(ids_birth_location))}", 1, 3)
prisma_obj.add_element(f"UK Biobank Population that can be geolocated at birth\n\nN = {sep_num(len(ids_birth_location))}", 0, 4)

# Construct a valid list of dates based on those born after june 1946; the introduction of penicillin
dates_list = [datetime(int(row[year_i]), int(row[month_i]), 1) for row in ids_birth_location]
valid_dates = [True if date > datetime(1946, 6, 1) else False for date in dates_list]

# We only include those after 1946
ids_sample_data = [row for row, valid in zip(ids_birth_location, valid_dates) if valid]
prisma_obj.add_element(f"Born before June 1946\n\nN = {sep_num(len(ids_birth_location) - len(ids_sample_data))}", 1, 5)
prisma_obj.add_element(f"UK Biobank Population post introduction of penicillin\n\nN = {sep_num(len(ids_sample_data))}", 0, 6)

# Create a lookup if ID: regional values for those whom pass the QC
ids_search_dict = {row[0]: row for row in ids_sample_data}

UK Biobank Population

N = 502 488
------------------------------
Born in Scotland

N = 39 488
------------------------------
UK Biobank Population not in
Scotland

N = 463 000
------------------------------
No or invalid birth
coordinates

N = 57 799
------------------------------
UK Biobank Population that can
be geolocated at birth

N = 405 201
------------------------------
Born before June 1946

N = 122 177
------------------------------
UK Biobank Population post
introduction of penicillin

N = 283 024
------------------------------


### Non Calender fixed effects

Given our data is looking at seasons of disease, which traditionally start in autumn it may be more logical to control
for time via fixed effects of the seasons of disease as apposed to calender dates. These alternative fixed effects are
September to August, rather than  January-December, in line with [Lamagni et al 2018][lam].

[lam]: https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(17)30693-X/fulltext


In [22]:
sep_august = [datetime(int(1930 + i), 9, 1) for i in range(45)]
sep_august_list = {row[0]: str(rebase_year(datetime(int(row[year_i]), int(row[month_i]), 1), sep_august))
                   for row in ids_sample_data}

print(f"Construct September August Fixed effects {terminal_time()}")

Construct September August Fixed effects 15:13


### Compute rates

Our phenotype exposure of interested is Scarlet Fever, which is currently just a count data type that has been 
aggregated from weeks into years. We want to calculate the risk of exposure, which means we need to account for the 
population of the district the individual was born in. This calculates the exposure at a given age between 1-10 of 
scarlet fever. 

To do this, we isolate all the unique birth days per district, then extract the exposures at each unique birth date 
before assigning them to individuals.

In [23]:
id_dates_birth = {row[0]: f"{row[year_i]}__{row[month_i]}__{row[district_id]}__{row[district_i]}{row[district_type_i]}"
                  for row in ids_sample_data}


unique_values = sorted(list(set([value for value in id_dates_birth.values()])))
print(f"There are {len(unique_values)} Unique Place-Date of birth values to exploit")

There are 111783 Unique Place-Date of birth values to exploit


In [24]:
bio_gis_data = load_json(Path(DATABASE_DIRECTORY, "Json", "Wave1Weighted_Districts.txt"))
print(f"Loaded BIO-HGIS Database {terminal_time()}")

Loaded BIO-HGIS Database 15:13


In [25]:
phenotypes = ["Scarlet_Fever"]
link_dict = assigned_exposure(unique_values, phenotypes, bio_gis_data, 10, [1, 5, 10])
clear_output(True)
print(f"Constructed Exposures {terminal_time()}")

Constructed Exposures 15:14


### Load supporting 

Here we load additional phenotypic data directory from the UK Biobank so we can assign it

In [26]:
cvd_file = CsvObject(Path(PROJECT_DIRECTORY, "CVD_Outcomes.csv"))
other_phenotypes = CsvObject(Path(PROJECT_DIRECTORY, "Phenotypes.csv"))
gender = CsvObject(Path(CONSTRUCTION_DIRECTORY, "Gender.csv"))

gender_lookup = {row[0]: row[1] for row in gender.row_data}
cvd_file_lookup = {row[0]: row[1:] for row in cvd_file.row_data}
other_phenotypes_lookup = {row[0]: row[1:] for row in other_phenotypes.row_data}
print(f"Set supporting {terminal_time()}")

Set supporting 15:14


### Remove those without 10 years of exposures

We will be looking at the average exposure across 10 years, but to ensure this is comparable we need to only include
those with exposures over 10 years

In [27]:
output_row = []

for ids, place in id_dates_birth.items():
    
    exposure_values = link_dict[place]
    
    if "NA" not in exposure_values:    
        output_row.append([ids_search_dict[ids][h] for h in [0, year_i, month_i, district_id, district_i, district_type_i]] +
                          [sep_august_list[ids]] +
                          [gender_lookup[ids]] +
                          other_phenotypes_lookup[ids] +
                          cvd_file_lookup[ids] +
                          exposure_values)

# We only include those whom we have 10 years of exposure data.
prisma_obj.add_element(f"Lacks 10 years of exposure data or born after 1963\n\nN = {sep_num(len(ids_sample_data) - len(output_row))}", 1, 7)
prisma_obj.add_element(f"UK Biobank Population Analysis Sample\n\nN = {sep_num(len(output_row))}", 0, 8)

# Write out PRISMA dict for pyBlendFigures
prisma_obj.write_prisma_config(prisma_obj.prisma_links(), BLEND_FIG_DIR, "Prisma")

# Construct headers and ensure that they are of equal length
headers = [id_data.headers[i] for i in [0, year_i, month_i, district_id, district_i, district_type_i]] + \
          ["SepYoB", "Gender"] + other_phenotypes.headers[1:] + cvd_file.headers[1:] +\
          [f"SF_{i}" for i in range(1, 11)] + [f"SFA_{i}" for i in [1, 5, 10]]

# Check the header length is equal to the number of rows
if len(headers) == len(output_row[0]):
    print(f"Headers length of {len(headers)} equals Row length of {len(output_row[0])}")
else:
    print(f"Headers length of {len(headers)} does not equals Row length of {len(output_row[0])}")

Lacks 10 years of exposure
data or born after 1963

N = 41 339
------------------------------
UK Biobank Population Analysis
Sample

N = 241 685
------------------------------
Headers length of 41 equals Row length of 41


In [28]:
write_csv(PROJECT_DIRECTORY, "ScarletLong", headers, output_row)
print(f"Written to Analysis sample to disk {terminal_time()}")

Written to Analysis sample to disk 15:20
